## 필요 라이브러리

In [1]:
# 크롬 브라우저 다루기 위함
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

from tqdm.auto import tqdm

# 데이터프레임 다루기 위함
import pandas as pd


# 경고 메시지를 무시
import warnings
warnings.filterwarnings(action='ignore')

## Selenium Crawling

In [2]:
# data 수집 기간

start_days = ['20200101', '20200401', '20200701', '20201001',\
             '20210101', '20210401', '20210701', '20211001',\
             '20220101', '20220401', '20220701', '20221001']
end_days =   ['20200331' ,'20200630', '20200930', '20201231',\
             '20210331' ,'20210630', '20210930', '20211231',\
             '20220331' ,'20220630', '20220930', '20221231'] 

In [4]:
# crawling 결과 저장 리스트
rows_list = []

# DataFrame 생성
cols_list = ['업무', '입찰공고번호', '재입찰', '공고명', '수요기관', '개찰일시', '참가수', '낙찰자', '낙찰금액(원)', '낙찰률(%)', '비고', '링크']
df = pd.DataFrame(columns=cols_list)

driver = webdriver.Chrome('./chromedriver_win32/chromedriver')

for i in tqdm(range(len(start_days))):
    driver.get('https://www.g2b.go.kr:8101/ep/tbid/tbidFwd.do?bidSearchType=1')

    # 최종낙찰자 체크
    checkbox = driver.find_element('id', 'bidSearchType3')
    checkbox.click()

    # 업무 종류 체크
    task_dict = {'물품':'taskClCds1','용역': 'taskClCds5'}
    for task in task_dict.values():
        checkbox = driver.find_element('id', task)
        checkbox.click()

    # 검색어
    query = '인공지능'
    # id값이 bidNm인 태그 가져오기
    bidNm = driver.find_element('id', 'bidNm')
    # 내용을 삭제 (버릇처럼 사용할 것!)
    bidNm.clear()
    # 검색어 입력후 엔터
    bidNm.send_keys(query)
    # bidNm.send_keys(Keys.RETURN)

    # 기간 설정
    start_day = start_days[i]
    end_day = end_days[i]

    sdate_box = driver.find_element('id', 'fromOpenBidDt')
    sdate_box.clear()
    sdate_box.send_keys(start_day)

    edate_box = driver.find_element('id', 'toOpenBidDt')
    edate_box.clear()
    edate_box.send_keys(end_day)

    # 목록수 100건 선택 (드롭다운)
    recordcountperpage = driver.find_element('id', 'recordCountPerPage')
    selector = Select(recordcountperpage)
    selector.select_by_value('100')

    # 검색 버튼 클릭
    search_button = driver.find_element(By.XPATH, '//*[@id="buttonwrap"]/div/a[1]')
    search_button.click()

    # 더보기가 존재하면 클릭
    try:
        more = driver.find_element(By.XPATH, '//*[@id="pagination"]/a')
        print(start_day, '-',end_day, '더보기 있음')
        if more:
            more.click()
    except:
        pass

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # bs4 이용한 로우추출
    rows = soup.select('#container > div > table > tbody > tr > td')
    for idx, val in enumerate(rows):
        # 상세조회가 포함된 row는 3칸의 빈칸 text가 추출이 안됨
        if val == '상세조회':
            rows_list.append('')
            rows_list.append('')
            rows_list.append('')
        rows_list.append(val.text.replace('\t','').replace('\n','').strip())

  0%|          | 0/12 [00:00<?, ?it/s]

## Data preprocessing

In [5]:
new_results = []
init_val = 0
last_idx = 0
for idx, val in enumerate(rows_list):
    if val == '물품' or val == '용역' :
        new_results.append(rows_list[init_val:idx])
        pre_len = len(rows_list[init_val:idx])
        init_val += pre_len
        last_idx = idx
new_results.append(rows_list[last_idx:last_idx+12])

In [6]:
new_results = new_results[1:]
new_results

[['용역',
  '20200101696-00',
  '0',
  '인공지능 기반 창의융합 교육 지원 시스템 구축 사업',
  '대구교육대학교',
  '2020/01/15 11:00',
  '2',
  '(주)자이닉스',
  '196,000,000',
  '',
  '',
  ''],
 ['물품',
  '20200103401-00',
  '0',
  '[인공지능학과]실험실습기자재 구입(고성능학습서버시스템 등 2종)',
  '경북대학교 융합교육지원센터',
  '2020/01/15 13:00',
  '22',
  '씨지에스아이티(주)',
  '116,999,990',
  '96.795',
  '',
  ''],
 ['물품',
  '20200109834-00',
  '0',
  '와이즈유(영산대학교) 인공지능 교육용 GPU 서버 구매',
  '영산대학교',
  '2020/01/21 13:00',
  '2',
  '(주)유진이아이에스',
  '70,000,000',
  '97.63',
  '',
  ''],
 ['용역',
  '20191233969-00',
  '0',
  '2020년 인공지능 고성능 컴퓨팅 자원 임차 용역',
  '정보통신산업진흥원',
  '2020/01/31 11:00',
  '2',
  '네이버비즈니스플랫폼 주식회사',
  '17,680,200,000',
  '',
  '',
  ''],
 ['용역',
  '20200107808-00',
  '0',
  '인공지능 학습 지원을 위한 의료 데이터(비정형) 자료 수집',
  '한국산업기술시험원',
  '2020/02/07 11:00',
  '2',
  '고려대학교산학협력단',
  '460,000,000',
  '',
  '',
  ''],
 ['용역',
  '20200117191-00',
  '0',
  '인공지능 학습 지원을 위한 의료 데이터(정형) 자료 수집',
  '한국산업기술시험원',
  '2020/02/07 11:00',
  '2',
  '아주대학교 산학협력단',
  '300,000,000'

In [7]:
for idx, val in enumerate(new_results):
    try:
        df = df.append(pd.Series(val, index=df.columns), ignore_index=True)
    except:
        pass

In [8]:
df['검색어'] = query

## Dataview  & save

In [9]:
df.tail()

,업무,입찰공고번호,재입찰,공고명,수요기관,개찰일시,참가수,낙찰자,낙찰금액(원),낙찰률(%),비고,링크,검색어
603,물품,20221227209-00,0,대구공업고등학교 (가칭)미래직업교육센터 기자재 구매(인공지능 휴머노이드 서비스 로봇...,대구광역시교육청 대구공업고등학교,2022/12/21 13:00,1,주식회사 제이엠로보틱스,"99,000,000",99.986,,,인공지능
604,물품,20221229760-00,0,순천청암고등학교 AI(인공지능) 현안사업 기자재 구입 수의견적 제출안내 재공고,전라남도교육청 순천청암고등학교,2022/12/22 11:00,8,만능유한회사,"27,159,270",98.284,,,인공지능
605,물품,20221234231-00,0,대구공업고등학교 (가칭)미래직업교육센터 기자재 구매(인공지능 교육용 블록 키트 외 4종),대구광역시교육청 대구공업고등학교,2022/12/22 13:00,1,주식회사 제이엠로보틱스,"180,500,000",99.89,,,인공지능
606,물품,20221233708-00,0,카메라와 인공지능을 활용한 스마트물꼬사업 물꼬 설치(4차),전라북도 김제시,2022/12/26 17:00,2,한국정밀농업 주식회사,"27,807,500",88.604,,,인공지능
607,물품,20221227129-00,0,금성고등학교 AI(인공지능) 교실 물품 구매(규격가격동시입찰),전라남도교육청 금성고등학교,2022/12/28 11:00,2,주식회사 이너스시스템,"84,700,000",91.763,,,인공지능


In [10]:
df.to_excel('./20-23_나라장터_최종낙찰자_인공지능(물품,용역).xlsx', index=False)